#HomeWork 3 - Mahima Regunathan#
Collaboration: Alex Rose

In [2]:
from pyspark.sql import SparkSession
#import pyspark.sql.functions as F
from pyspark.sql import Row, Column, Window
from pyspark.sql.types import *

**Task 1**

In [4]:
from pyspark.sql.functions import sum

In [6]:
display(dbutils.fs.ls("dbfs:/"))

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/local_disk0/,local_disk0/,0
dbfs:/ml/,ml/,0
dbfs:/tmp/,tmp/,0


In [7]:
dbutils.fs.ls("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv")

Out[ 2 ]: [FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv', name='2010-summary.csv', size=7121),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv', name='2011-summary.csv', size=7069),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv', name='2012-summary.csv', size=6857),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv', name='2013-summary.csv', size=7020),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv', name='2014-summary.csv', size=6729),
 FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv', name='2015-summary.csv', size=7080)]

**Task 2**

In [9]:
flights2010DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv", header = True)
flights2011DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2011-summary.csv", header = True)
flights2012DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2012-summary.csv", header = True)
flights2013DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2013-summary.csv", header = True)
flights2014DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2014-summary.csv", header = True)
flights2015DF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv", header = True)

In [10]:
flightsDF = flights2010DF.union(flights2011DF)
flightsDF = flightsDF.union(flights2012DF)
flightsDF = flightsDF.union(flights2013DF)
flightsDF = flightsDF.union(flights2014DF)
flightsDF = flightsDF.union(flights2015DF)

In [11]:
flightsDF.count()

Out[ 5 ]: 1502

**Task 3**

In [13]:
# Reading all the csv files with default value for inferSchema, which is False
flightsAllDF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", header = True)

In [14]:
# Reading all the csv files with passing inferSchema as True
flightsAllDF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", inferSchema = True, header = True)

In [15]:
flightsAllDF.count()

Out[ 8 ]: 1502

**Task 4**

In [17]:
# Defining the input schema
myschema = StructType([
StructField("dest", StringType()),\
StructField("origin", StringType()),\
StructField("count", LongType())
])

In [18]:
# Reading the csv files using the schema defined above
flightsAllDF = spark.read.csv("dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/*.csv", schema = myschema, header = True)

**Task 5**

In [20]:
flightsAllDF.count()

Out[ 11 ]: 1502

**Task 6**

In [22]:
# Method 1 to print the first 3 records
flightsAllDF.head(3)

Out[ 12 ]: [Row(dest='United States', origin='Romania', count=1),
 Row(dest='United States', origin='Ireland', count=264),
 Row(dest='United States', origin='India', count=69)]

In [23]:
# Method 2 to print the first 3 records
flightsAllDF.show(3)

+-------------+-------+-----+
 dest| origin|count|
+-------------+-------+-----+
United States|Romania| 1|
United States|Ireland| 264|
United States| India| 69|
+-------------+-------+-----+
only showing top 3 rows

In [24]:
# Method 3 to print the first 3 records
flightsAllDF.take(3)

Out[ 14 ]: [Row(dest='United States', origin='Romania', count=1),
 Row(dest='United States', origin='Ireland', count=264),
 Row(dest='United States', origin='India', count=69)]

**Task 7**

In [26]:
flightsAllDF.columns

Out[ 15 ]: ['dest', 'origin', 'count']

In [27]:
# Adding a new column "filename" and populating it with the file name
flightsAllDF = flightsAllDF.withColumnRenamed("count", "numflights")\
  .withColumn("filename", F.input_file_name())

# flightsAllDF = flightsAllDF.withColumnRenamed("count", "numflights")\
# .withColumn("filename", F.regexp_extract(F.input_file_name(), "[ \w-]+?(?=\.)", 0))


In [28]:
flightsAllDF.columns

Out[ 17 ]: ['dest', 'origin', 'numflights', 'filename']

In [29]:
# flightsAllDF.select(["filename"]).head(5)
flightsAllDF.select(["filename"]).show(5)

+--------------------+
 filename|
+--------------------+
dbfs:/databricks-...|
dbfs:/databricks-...|
dbfs:/databricks-...|
dbfs:/databricks-...|
dbfs:/databricks-...|
+--------------------+
only showing top 5 rows

In [30]:
display(flightsAllDF.select(["filename"]).limit(5))

filename
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv
dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2010-summary.csv


**Task 8**

In [32]:
# Adding a new column "year" and populating it with the year part in the filename
str = "234Mahima"
newstr = F.regexp_extract(str, "^[0-9]{2,3}[a-zA-Z]?", 0)
newstr

Out[ 21 ]: Column<b'regexp_extract(234Mahima, ^[0-9]{2,3}[a-zA-Z]?, 0)'>

In [33]:
flightsAllDF.show(5)

+-----------------+-------------+----------+----+
 dest| origin|numflights|year|
+-----------------+-------------+----------+----+
 United States| Romania| 1|2010|
 United States| Ireland| 264|2010|
 United States| India| 69|2010|
 Egypt|United States| 24|2010|
Equatorial Guinea|United States| 1|2010|
+-----------------+-------------+----------+----+
only showing top 5 rows

**Task 9**

In [35]:
# flightsAllDF.sort("numflights", ascending = False).show(5)
# Sorting the df by numflights in desc order
flightsAllDF\
  .sort(F.desc("numflights"))\
  .show(5)

+-------------+-------------+----------+----+
 dest| origin|numflights|year|
+-------------+-------------+----------+----+
United States|United States| 370002|2015|
United States|United States| 358354|2014|
United States|United States| 352742|2011|
United States|United States| 348113|2010|
United States|United States| 347452|2012|
+-------------+-------------+----------+----+
only showing top 5 rows

**Task 10**

** a. Count of Distinct destinations **

In [38]:
flightsAllDF\
  .select("dest")\
  .distinct()\
  .count()

Out[ 23 ]: 167

** b. Count of distinct Destination & Year **

In [40]:
flightsAllDF\
  .select(["dest", "year"])\
  .distinct()\
  .count()

Out[ 24 ]: 772

**Task 11**

**a**

In [43]:
flightsAllDF\
  .where(F.col("dest").isin(["Japan", "South Korea"]))\
  .count()

Out[ 25 ]: 12

In [44]:
flightsAllDF\
  .where((F.col("dest")=="Japan") | (F.col("dest")=="South Korea"))\
  .count()

Out[ 26 ]: 12

**b**

In [46]:
flightsAllDF\
  .where((F.col("origin")=="United States") & (F.col("dest")=="South Korea") & (F.col("year") == "2013"))\
  .show()

+-----------+-------------+----------+----+
 dest| origin|numflights|year|
+-----------+-------------+----------+----+
South Korea|United States| 842|2013|
+-----------+-------------+----------+----+

Flights from United States to South Korea in 2013: 842

**Task 12**

In [49]:
flightsAllDF\
  .where(F.col("origin") == F.col("dest"))\
  .sort(["year", "numflights"])\
  .show()

+-------------+-------------+----------+----+
 dest| origin|numflights|year|
+-------------+-------------+----------+----+
United States|United States| 348113|2010|
United States|United States| 352742|2011|
United States|United States| 347452|2012|
United States|United States| 343132|2013|
United States|United States| 358354|2014|
United States|United States| 370002|2015|
+-------------+-------------+----------+----+

In [50]:
display(flightsAllDF.where(F.col("origin") == F.col("dest")).sort(["year", "numflights"]))

dest,origin,numflights,year
United States,United States,348113,2010
United States,United States,352742,2011
United States,United States,347452,2012
United States,United States,343132,2013
United States,United States,358354,2014
United States,United States,370002,2015


**Task 13**

In [52]:
flightsAllDF\
  .groupby()\
  .sum("numflights")\
  .show(truncate=False)

+---------------+
sum(numflights)|
+---------------+
2580915 |
+---------------+

**Task 14**

In [54]:
flightsAllDF\
  .where((F.col("dest") == "United States") & (F.col("origin") != "United States"))\
  .groupby("year")\
  .sum("numflights")\
  .withColumnRenamed("sum(numflights)", "totalflightsin")\
  .sort("year")\
  .show()

+----+--------------+
year|totalflightsin|
+----+--------------+
2010| 36819|
2011| 37763|
2012| 36890|
2013| 36830|
2014| 38833|
2015| 41350|
+----+--------------+

**Task 15**

In [56]:
flightsin = flightsAllDF\
  .where((F.col("dest") == "United States") & (F.col("origin") != "United States"))\
  .groupby("year")\
  .sum("numflights")\
  .withColumnRenamed("sum(numflights)", "totalflightsin")

flightsout = flightsAllDF\
  .where((F.col("origin") == "United States") & (F.col("dest") != "United States"))\
  .groupby("year")\
  .sum("numflights")\
  .withColumnRenamed("sum(numflights)", "totalflightsout")

flights = flightsin.join(flightsout, on="year")

display(flights.withColumn("morein", F.col("totalflightsin") > F.col("totalflightsout")))


year,totalflightsin,totalflightsout,morein
2015,41350,41964,false
2013,36830,37907,false
2014,38833,39668,false
2012,36890,37838,false
2010,36819,37337,false
2011,37763,37921,false


**Task 16**

In [58]:
flightsAllDF.where((F.col("dest") == "United States") & (F.col("origin") != "United States"))\
  .groupby("year")\
  .agg(sum("numflights").alias("sumflights"), F.avg("numflights").alias("avgflights"), F.max("numflights").alias("maxflights"))\
  .sort("year")\
  .show()

+----+----------+------------------+----------+
year|sumflights| avgflights|maxflights|
+----+----------+------------------+----------+
2010| 36819| 283.2230769230769| 8305|
2011| 37763| 299.7063492063492| 8650|
2012| 36890| 312.6271186440678| 8097|
2013| 36830| 320.2608695652174| 7983|
2014| 38833| 331.9059829059829| 8177|
2015| 41350|333.46774193548384| 8483|
+----+----------+------------------+----------+

**Task 17**

In [60]:
# countDistinct()
# collect_set()
unique_dest = flightsAllDF\
  .where((F.col("origin") == "United States") & (F.col("dest") != "United States")).groupby("year")\
  .agg(F.countDistinct("dest").alias("count of unique dest"), F.collect_set("dest").alias("unique dest"))

unique_dest.show()

+----+--------------------+--------------------+
year|count of unique dest| unique dest|
+----+--------------------+--------------------+
2015| 131|[Italy, Martiniqu...|
2013| 134|[Italy, Martiniqu...|
2014| 123|[Italy, Slovakia,...|
2012| 126|[Italy, Martiniqu...|
2010| 124|[Italy, Martiniqu...|
2011| 128|[Italy, Martiniqu...|
+----+--------------------+--------------------+

In [61]:
display(unique_dest.sort("count of unique dest", ascending = False).head(1))

year,count of unique dest,unique dest
2013,134,"List(Italy, Martinique, Finland, Germany, Kiribati, French Guiana, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Russia, Japan, Federated States of Micronesia, Guadeloupe, Burundi, Saint Kitts and Nevis, Equatorial Guinea, Aruba, South Africa, New Zealand, Kuwait, Tanzania, Rwanda, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Saint Lucia, The Bahamas, Nepal, Greece, Bermuda, Taiwan, Spain, Kazakhstan, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Jordan, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Indonesia, Nicaragua, New Caledonia, Greenland, Costa Rica, Peru, South Korea, Portugal, Thailand, French Polynesia, Belize, Ghana, Palau, Hungary, Paraguay, Sierra Leone, Antigua and Barbuda, Barbados, Algeria, Saint Barthelemy, Senegal, United Kingdom, Libya, Nigeria, Guyana, Iceland, Cayman Islands, Lithuania, Bahrain, Panama, Macau, Kyrgyzstan, Ukraine, Guatemala, Morocco, Netherlands, Niger, Trinidad and Tobago, Brunei, Norway, Anguilla, Malaysia, Bolivia, Samoa, Israel, Sint Maarten, Papua New Guinea, Kenya, Zimbabwe, Canada, Ecuador, Switzerland, Grenada, Singapore, El Salvador, Cuba, Hong Kong, Lebanon, Cyprus, India, Denmark, China, Bonaire, Sint Eustatius, and Saba, Philippines, Mexico, Haiti, Poland, Brazil, Chile, Saint Helena, Ascension, and Tristan da Cunha, France, Saint Vincent and the Grenadines, Egypt, Curacao, Qatar)"


**Task 18**

In [63]:
# unique origin - countries that sent flights to us
# unique dest - contries received flights from us - from Task 17
# merge the two dataframe on year
unique_origin = flightsAllDF\
  .where((F.col("dest") == "United States") & (F.col("origin") != "United States"))\
  .groupby("year")\
  .agg(F.countDistinct("origin").alias("count of unique origin"), F.collect_set("origin").alias("unique origin"))

display(unique_origin)

year,count of unique origin,unique origin
2015,124,"List(Italy, Martinique, Liberia, Finland, Germany, Kiribati, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Namibia, Japan, Russia, Federated States of Micronesia, Guadeloupe, Bulgaria, Saint Kitts and Nevis, Malta, Aruba, South Africa, New Zealand, Kuwait, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Bermuda, Saint Lucia, The Bahamas, Greece, Taiwan, Spain, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Montenegro, Jordan, Vietnam, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Indonesia, Nicaragua, Greenland, Costa Rica, Peru, South Korea, Thailand, Portugal, Belize, French Polynesia, Ghana, Palau, Hungary, Paraguay, Antigua and Barbuda, Saint Barthelemy, Barbados, Senegal, United Kingdom, Nigeria, Guyana, Iceland, Cayman Islands, Lithuania, Bahrain, Panama, Ukraine, Guatemala, Morocco, Netherlands, Trinidad and Tobago, Malaysia, Anguilla, Bolivia, Norway, Croatia, Samoa, Israel, Sint Maarten, Gibraltar, Papua New Guinea, Canada, Grenada, Switzerland, Ecuador, Singapore, Hong Kong, El Salvador, Cuba, Georgia, Cyprus, India, Denmark, Estonia, China, Bonaire, Sint Eustatius, and Saba, Philippines, Haiti, Mexico, Poland, Brazil, Chile, France, Egypt, Curacao, Qatar)"
2013,115,"List(Italy, Martinique, Finland, Germany, Kiribati, French Guiana, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Russia, Japan, Federated States of Micronesia, Guadeloupe, Saint Kitts and Nevis, Aruba, South Africa, New Zealand, Kuwait, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Saint Lucia, The Bahamas, Bermuda, Greece, Taiwan, Spain, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Jordan, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Cambodia, Nicaragua, Greenland, Costa Rica, Peru, South Korea, Portugal, French Polynesia, Belize, Ghana, Palau, Paraguay, Antigua and Barbuda, Barbados, Saint Barthelemy, Senegal, United Kingdom, Nigeria, Guyana, Iceland, Cayman Islands, Bahrain, Panama, Ukraine, Guatemala, Morocco, Mali, Netherlands, Niger, Trinidad and Tobago, Norway, Anguilla, Malaysia, Bolivia, Croatia, Samoa, Israel, Sint Maarten, Zimbabwe, Canada, Ecuador, Switzerland, Grenada, Singapore, El Salvador, Cuba, Hong Kong, India, Denmark, China, Bonaire, Sint Eustatius, and Saba, Philippines, Mexico, Haiti, Poland, Brazil, Chile, Saint Helena, Ascension, and Tristan da Cunha, France, Saint Vincent and the Grenadines, Egypt, Curacao, Qatar)"
2014,117,"List(Italy, Martinique, Finland, Germany, French Guiana, Kiribati, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Russia, Japan, Federated States of Micronesia, Guadeloupe, Bulgaria, Saint Kitts and Nevis, Malta, Aruba, South Africa, New Zealand, Kuwait, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Saint Lucia, Bermuda, The Bahamas, Greece, Taiwan, Spain, Kazakhstan, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Jordan, Austria, Colombia, Ireland, Romania, Saudi Arabia, Latvia, Fiji, Indonesia, Cambodia, Nicaragua, Greenland, Costa Rica, Peru, South Korea, Portugal, French Polynesia, Belize, Ghana, Palau, Hungary, Paraguay, Antigua and Barbuda, Barbados, Saint Barthelemy, United Kingdom, Senegal, Nigeria, Guyana, Iceland, Cayman Islands, Panama, Ukraine, Guatemala, Morocco, Netherlands, Trinidad and Tobago, Norway, Bolivia, Anguilla, Croatia, Samoa, Israel, Sint Maarten, Saint Martin, Canada, Ecuador, Switzerland, Grenada, Hong Kong, Cuba, El Salvador, India, Denmark, Congo (Brazaville), China, Bonaire, Sint Eustatius, and Saba

In [64]:
merged_df = unique_dest.join(unique_origin, on="year")
display(merged_df)

year,count of unique dest,unique dest,count of unique origin,unique origin
2015,131,"List(Italy, Martinique, Liberia, Finland, Germany, French Guiana, Kiribati, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Burkina Faso, Venezuela, Japan, Russia, Federated States of Micronesia, Guadeloupe, Bulgaria, Saint Kitts and Nevis, Malta, Aruba, South Africa, New Zealand, Kuwait, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Bermuda, Saint Lucia, The Bahamas, Greece, Taiwan, Spain, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Jordan, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Djibouti, Kosovo, Indonesia, Nicaragua, New Caledonia, Greenland, Costa Rica, Peru, South Korea, Thailand, Portugal, Belize, French Polynesia, Ghana, Palau, Hungary, Cote d'Ivoire, Paraguay, Antigua and Barbuda, Saint Barthelemy, Barbados, Algeria, Iraq, Senegal, United Kingdom, Nigeria, Guyana, Iceland, Cayman Islands, Bahrain, Panama, Ukraine, Guatemala, Morocco, Netherlands, Niger, Trinidad and Tobago, Malaysia, Anguilla, Bolivia, Norway, Croatia, Zambia, Samoa, Israel, Sint Maarten, Papua New Guinea, Moldova, Canada, Grenada, Switzerland, Ecuador, Singapore, Hong Kong, El Salvador, Cuba, Georgia, Cyprus, India, Denmark, China, Bonaire, Sint Eustatius, and Saba, Philippines, Haiti, Mexico, Poland, Brazil, Chile, France, Saint Vincent and the Grenadines, Tunisia, Egypt, Curacao, Qatar)",124,"List(Italy, Martinique, Liberia, Finland, Germany, Kiribati, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Namibia, Japan, Russia, Federated States of Micronesia, Guadeloupe, Bulgaria, Saint Kitts and Nevis, Malta, Aruba, South Africa, New Zealand, Kuwait, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Bermuda, Saint Lucia, The Bahamas, Greece, Taiwan, Spain, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Montenegro, Jordan, Vietnam, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Indonesia, Nicaragua, Greenland, Costa Rica, Peru, South Korea, Thailand, Portugal, Belize, French Polynesia, Ghana, Palau, Hungary, Paraguay, Antigua and Barbuda, Saint Barthelemy, Barbados, Senegal, United Kingdom, Nigeria, Guyana, Iceland, Cayman Islands, Lithuania, Bahrain, Panama, Ukraine, Guatemala, Morocco, Netherlands, Trinidad and Tobago, Malaysia, Anguilla, Bolivia, Norway, Croatia, Samoa, Israel, Sint Maarten, Gibraltar, Papua New Guinea, Canada, Grenada, Switzerland, Ecuador, Singapore, Hong Kong, El Salvador, Cuba, Georgia, Cyprus, India, Denmark, Estonia, China, Bonaire, Sint Eustatius, and Saba, Philippines, Haiti, Mexico, Poland, Brazil, Chile, France, Egypt, Curacao, Qatar)"
2013,134,"List(Italy, Martinique, Finland, Germany, Kiribati, French Guiana, Dominica, Sweden, Angola, Belgium, Ethiopia, United Arab Emirates, Australia, Cook Islands, Venezuela, Russia, Japan, Federated States of Micronesia, Guadeloupe, Burundi, Saint Kitts and Nevis, Equatorial Guinea, Aruba, South Africa, New Zealand, Kuwait, Tanzania, Rwanda, Azerbaijan, Jamaica, Turks and Caicos Islands, British Virgin Islands, Turkey, Uruguay, Marshall Islands, Czech Republic, Pakistan, Saint Lucia, The Bahamas, Nepal, Greece, Bermuda, Taiwan, Spain, Kazakhstan, Luxembourg, Dominican Republic, Suriname, Honduras, Cape Verde, Argentina, Jordan, Austria, Ireland, Colombia, Romania, Saudi Arabia, Latvia, Fiji, Indonesia, Nicaragua, New Caledonia, Greenland, Costa Rica, Peru, South Korea, Portugal, Thailand, French Polynesia, Belize, Ghana, Palau, Hungary, Paraguay, Sierra Leone, Antigua and Barbuda, Barbados, Algeria, Saint Barthelemy, Senegal, United Kingdom, Libya, Nigeria, Guyana, Iceland, Cayman Islands, Lithuania, Bahrain, Panama, Macau, Kyrgyzstan, Ukrain

In [65]:
# User defined function to Set Difference
# Register the user defined function with spark
@udf()
def setDifference(a, b):
  return sorted(set(a) - set(b))

# Register the udf with Spark
#spark.udf.register("setDifference", setDifference)

In [66]:
display(merged_df.select("year", setDifference(F.col("unique dest"), F.col("unique origin")).alias("receivedontsend")).sort("year"))

year,receivedontsend
2010,"[Equatorial Guinea, Georgia, Kazakhstan, Paraguay]"
2011,"[Belarus, Bulgaria, Lebanon, Libya, Macau, Namibia, The Gambia, Togo, Vietnam, Yemen]"
2012,"[Belarus, Bulgaria, Cameroon, Georgia, Hungary, Kazakhstan, Kyrgyzstan, Macau, Mauritania, New Caledonia, Oman, Sierra Leone, Vietnam]"
2013,"[Algeria, Azerbaijan, Brunei, Burundi, Cyprus, Equatorial Guinea, Hungary, Indonesia, Kazakhstan, Kenya, Kyrgyzstan, Lebanon, Libya, Lithuania, Macau, Nepal, New Caledonia, Papua New Guinea, Rwanda, Sierra Leone, Tanzania, Thailand]"
2014,"[Algeria, Bahrain, Burkina Faso, Chad, Gibraltar, Liberia, Macau, Malawi, Malaysia, Mauritania, Slovakia]"
2015,"[Algeria, Burkina Faso, Cote d'Ivoire, Djibouti, French Guiana, Iraq, Kosovo, Moldova, New Caledonia, Niger, Saint Vincent and the Grenadines, Tunisia, Zambia]"


**Task 19**

In [68]:
# partitionby("year")
# select(F.max().over(window))

# Create a window that is partitioned by year
window_year = Window.partitionBy("year")

In [69]:
windowsDF = flightsAllDF\
  .where((F.col("origin") == "United States") & (F.col("dest") != "United States"))\
  .select("dest", "origin", "numflights", "year", F.max("numflights").over(window_year).alias("maxflightstoanydest"))
  
windowsDF.count()

Out[ 27 ]: 766

In [70]:
windowsDFNew = windowsDF\
  .where(F.col("numflights") == F.col("maxflightstoanydest"))\
  .sort("year")

display(windowsDFNew)

dest,origin,numflights,year,maxflightstoanydest
Canada,United States,8271,2010,8271
Canada,United States,8514,2011,8514
Canada,United States,8034,2012,8034
Canada,United States,7860,2013,7860
Canada,United States,7974,2014,7974
Canada,United States,8399,2015,8399


In [71]:
display(windowsDFNew)

dest,origin,numflights,year,maxflightstoanydest
Canada,United States,8271,2010,8271
Canada,United States,8514,2011,8514
Canada,United States,8034,2012,8034
Canada,United States,7860,2013,7860
Canada,United States,7974,2014,7974
Canada,United States,8399,2015,8399


**Task 20**

In [73]:

pivotDF = flightsAllDF.where(F.col("origin").contains("land"))\
  .groupby("origin")\
  .pivot("year")\
  .agg(F.sum("numflights"))\
  .na.fill(0)\
  .select("*", F.greatest("2010", "2011", "2012", "2013", "2014", "2015").alias("highest"))
  
pivotDF.show()

+--------------------+----+----+----+----+----+----+-------+
 origin|2010|2011|2012|2013|2014|2015|highest|
+--------------------+----+----+----+----+----+----+-------+
Turks and Caicos ...| 147| 177| 183| 178| 204| 236| 236|
British Virgin Is...| 47| 65| 78| 94| 101| 80| 101|
 Finland| 20| 32| 21| 15| 19| 28| 32|
 Marshall Islands| 44| 49| 30| 33| 35| 39| 49|
 Cayman Islands| 251| 262| 247| 274| 278| 310| 310|
Netherlands Antilles| 289| 0| 0| 0| 0| 0| 289|
 Ireland| 264| 268| 252| 266| 291| 344| 344|
 Thailand| 13| 14| 6| 0| 0| 4| 14|
 Iceland| 129| 138| 150| 151| 177| 202| 202|
 Switzerland| 334| 319| 282| 285| 300| 305| 334|
 Cook Islands| 12| 12| 12| 12| 12| 13| 13|
 New Zealand| 71| 77| 65| 77| 77| 74| 77|
 Greenland| 2| 2| 1| 3| 1| 4| 4|
 Poland| 61| 42| 40| 37| 33| 33| 61|
 Solomon Islands| 0| 0| 1| 0| 0| 0| 1|
 Netherlands| 570| 622| 607| 618| 702| 660| 702|
+--------------------+----+----+----+----+----+----+-------+

In [74]:
display(pivotDF)

origin,2010,2011,2012,2013,2014,2015,highest
Turks and Caicos Islands,147,177,183,178,204,236,236
British Virgin Islands,47,65,78,94,101,80,101
Finland,20,32,21,15,19,28,32
Marshall Islands,44,49,30,33,35,39,49
Cayman Islands,251,262,247,274,278,310,310
Netherlands Antilles,289,0,0,0,0,0,289
Ireland,264,268,252,266,291,344,344
Thailand,13,14,6,0,0,4,14
Iceland,129,138,150,151,177,202,202
Switzerland,334,319,282,285,300,305,334
